In [ ]:
import pandas as pd
import pandas as pd
import os
import multiprocessing

import numbers
import datetime
import dateutil.parser
import re

In [ ]:
#Path to expanded tables
dir_ = 'output-data/expanded-tables/'

In [ ]:
#Existing English Tables
existing = open("output-data/english_table_names.txt", 'r')
existing_english_tables = [line.replace('\n', '') for line in existing.readlines()]
len(existing_english_tables)

In [ ]:
#New English Tables
new_english_tables = os.listdir('output-data/new-english-tables/')
len(new_english_tables)

## Statistics about the expanded English tables

In [ ]:
def get_table_statistics(file_name):
    
    if file_name in existing_english_tables:
        file = 'output-data/expanded-tables/' + file_name
    else:
        file = 'output-data/new-english-tables/' + file_name
    
    try:
        df = pd.read_json(file, compression='gzip', lines=True)
        
        number_of_rows = len(df.index)
        column_count = len(df.columns)
        empty_cells = df.isna().sum().sum()
        total_cells = number_of_rows * column_count

        column_name_and_density = {}
        overall_table_density = int((total_cells - empty_cells)/total_cells *100)

        for index, column in df.isna().sum().iteritems():
            column_name_and_density[index] = int(((number_of_rows - column) / number_of_rows) * 100)
    
        return [ file_name, number_of_rows, column_count, column_name_and_density, overall_table_density ]

    except ValueError:
        print(file_name)

In [ ]:
tables = existing_english_tables + new_english_tables
pool = multiprocessing.Pool(processes=30)
res = pool.map(get_table_statistics, tables)
pool.close()
pool.join()

In [ ]:
r = [re for re in res if re]
statistics = pd.DataFrame(r, columns=['file_name', 'number_of_rows', 'column_count', 'column_name_and_density', 'overall_table_density'])
statistics

In [ ]:
statistics.to_csv('output-data/statistics/expanded_tables_statistics.csv', index=False)

### Generate labels for columns

In [ ]:
statistics = pd.read_csv('output-data/statistics/expanded_tables_statistics.csv')

In [ ]:
labels = pd.read_csv('data/Final CTA and CPA Labels.csv')
labels

In [ ]:
#column name to CPA label
column_cpa = {}
for index, row in labels.iterrows():
    if row['column_name'] == 'name':
        column_cpa[row['column_name']] = 'name'
    else:
        column_cpa[row['column_name']] = row['CPA label']     

In [ ]:
#column name to CTA label
column_cta = {}
for index, row in labels.iterrows():
    if row['column_name'] == 'name':
        column_cta[row['class']+row['column_name']] = row['CTA label']
    else:
        column_cta[row['column_name']] = row['CTA label']

In [ ]:
#fallback columns: if there are multiple cta labels possible
column_fallback = {}
for index, row in labels.iterrows():
     if not pd.isnull(row['fallback_CTA_label']):
        column_fallback[row['column_name']] = row['fallback_CTA_label']

# manual correction
column_fallback['doortime'] = 'DateTime'

In [ ]:
#Minimum 3 columns
tables_dict = statistics.loc[ statistics['column_count'] >=3 ].to_dict('records')
len(tables_dict)

tables = {}
for row in tables_dict:
    tables[row['file_name']] = row

#Existing English Tables
existing = open("output-data/english_table_names.txt", 'r')
existing_english_tables = [line.replace('\n', '') for line in existing.readlines()]
len(existing_english_tables)

In [ ]:
time_format = 'T*[0-9]{2}:[0-9]{2}:*[0-9]{0,2}'
months = ['jan', 'feb', 'mar', 'may', 'apr', 'june', 'july', 'sep', 'oct', 'nov', 'dec', 'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'septmeber', 'october', 'november', 'december']

In [ ]:
#what properties do the expected schema.org types have
types_dict = {}

for cta in column_cta:
    if ', ' in column_cta[cta]:
        types = column_cta[cta].split(', ')
        
        for t in types:
            df = pd.read_csv('data/PropsToTypes/'+t+'_propsToTypes.csv')
            df['prop'] = df['property'].apply(lambda row: row.split('.')[1].lower())
            props = df['prop'].tolist()
            types_dict[t] = props


In [ ]:
def get_dict_type(row, column_types, column_name, type_counts):
    
    #Each index represents a Type, each number represents the number of overlapping properties to a type
    find_match = []
                    
    for column_type in column_types:
        #How many row properties are in common with each expected column type:
        find_match.append( len( [prop for prop in list(row.keys()) if prop in types_dict[column_type] ]) )


    #If find match are equal: add 1 count to all types
    if all(x==find_match[0] for x in find_match):
        if 'ItemList' in column_types:
            for column_type in column_types:
                if column_type != 'ItemList':
                    if column_type not in type_counts:
                        type_counts[column_type] = 0
                    type_counts[column_type] += 1
        
        else:
            
            for column_type in column_types:
                if column_type not in type_counts:
                    type_counts[column_type] = 0
                type_counts[column_type] += 1

    else:
        #1 count to majority type and 0 for all others
        if column_types[find_match.index(max(find_match))] not in type_counts:
            type_counts[column_types[find_match.index(max(find_match))]] = 0

        type_counts[column_types[find_match.index(max(find_match))]] += 1

    return type_counts

In [ ]:
def get_list_of_dict_type(row, column_types, column_name, type_counts):

    row_type = []
    list_type_counts = {}
    
    #For element in row-list:
    for element in row:
        
        if isinstance(element, dict):
            list_type_counts = get_dict_type(element, column_types, column_name, list_type_counts) 
        elif isinstance(element, str):
            list_type_counts = get_str_type(element, column_types, column_name, list_type_counts)
        elif isinstance(element, list):
            list_type_counts = get_list_of_dict_type(element, column_types, column_name, list_type_counts)
        elif pd.isnull(element):
            if 'ItemList' in column_types:
                if 'ItemList' not in list_type_counts:
                    list_type_counts['ItemList'] = 0
                list_type_counts['ItemList'] += 1
            else:
                for column_type in column_types:
                    if column_type not in list_type_counts:
                        list_type_counts[column_type] = 0
                    list_type_counts[column_type] += 1
    
    
    if all(x==list(list_type_counts.values())[0] for x in list(list_type_counts.values())) and len(list_type_counts) > 1:
        if 'ItemList' in column_types:
            for column_type in column_types:
                if column_type == 'ItemList':
                    list_type_counts[column_type] = 0
                else:
                    list_type_counts[column_type] = 1
        else:
            for column_type in column_types:
                list_type_counts[column_type] = 1
    elif len(list_type_counts) > 1:
        for column_type in column_types:
            if column_type == max(list_type_counts, key=list_type_counts.get):
                list_type_counts[column_type] = 1
            else:
                list_type_counts[column_type] = 0
    else:
        if list_type_counts[max(list_type_counts, key=list_type_counts.get)] > 0:
            list_type_counts[max(list_type_counts, key=list_type_counts.get)] = 1

    for found_types in list_type_counts:
        if list_type_counts[found_types]:
            if found_types in type_counts:
                type_counts[found_types] += 1
            else:
                type_counts[found_types] = 1
    
    return type_counts            

In [ ]:
def get_str_type(row, column_types, column_name, type_counts):

    #If row is a number
    if isinstance(row, numbers.Number):
        if 'Number' not in column_types:
            if (isinstance(row, int) or row.is_integer()) and 'Integer' in column_types:
                if 'Integer' not in type_counts:
                    type_counts['Integer'] = 0
                type_counts['Integer'] += 1
                
            elif 'Text' in column_types:
                if 'Text' not in type_counts:
                    type_counts['Text'] = 0
                type_counts['Text'] += 1

            elif 'Date' in column_types:
                if 'Date' not in type_counts:
                    type_counts['Date'] = 0
                type_counts['Date'] += 1

            else:
                for column_type in column_types:
                    if column_type not in type_counts:
                        type_counts[column_type] = 0
                    type_counts[column_type] += 1
        else:
            if 'Number' not in type_counts:
                type_counts['Number'] = 0
            type_counts['Number'] += 1
    else:
        try:
            #If row is Date/DateTime/Time
            if isinstance(dateutil.parser.parse(row), datetime.date):
                row_date = dateutil.parser.parse(row)

                if re.match(time_format, row) and 'Time' in column_types:
                    if 'Time' not in type_counts:
                        type_counts['Time'] = 0
                    type_counts['Time'] += 1
                elif ':' in row and 'DateTime' in column_types:
                    if 'DateTime' not in type_counts:
                        type_counts['DateTime'] = 0
                    type_counts['DateTime'] += 1
                elif ':' in row and 'Date' in column_types :
                    if 'Date' not in type_counts:
                        type_counts['Date'] = 0
                    type_counts['Date'] += 1
                elif ':' not in row and 'Date' in column_types:
                    if 'Date' not in type_counts:
                        type_counts['Date'] = 0
                    type_counts['Date'] += 1
                elif ':' not in row and 'DateTime' in column_types:
                    if 'DateTime' not in type_counts:
                        type_counts['DateTime'] = 0
                    type_counts['DateTime'] += 1
                elif 'Text' in column_types:
                    if 'Text' not in type_counts:
                        type_counts['Text'] = 0
                    type_counts['Text'] += 1
                else:
                    if 'Wrong' not in type_counts:
                        type_counts['Wrong'] = 0
                    type_counts['Wrong'] += 1

        except Exception:
            #if row is a string
            if 'Date' in column_types or 'DateTime' in column_types or 'Time' in column_types:            
                if ':' in row.lower() or 'pm' in row.lower() or 'am' in row.lower():
                    if any(month in row.lower() for month in months) and 'DateTime' in column_types:
                        if 'DateTime' not in type_counts:
                            type_counts['DateTime'] = 0
                        type_counts['DateTime'] += 1
                    elif 'Time' in column_types:
                        if 'Time' not in type_counts:
                            type_counts['Time'] = 0
                        type_counts['Time'] += 1
                    elif 'Date' in column_types:
                        if 'Date' not in type_counts:
                            type_counts['Date'] = 0
                        type_counts['Date'] += 1
                elif 'Date' in column_types:
                    if 'Date' not in type_counts:
                        type_counts['Date'] = 0
                    type_counts['Date'] += 1
                elif 'DateTime' in column_types:
                    if 'DateTime' not in type_counts:
                        type_counts['DateTime'] = 0
                    type_counts['DateTime'] += 1

            elif 'URL' in column_types:
                if (re.search('\/[\w_-]+\/', row) or 'http' in row):
                    if 'URL' not in type_counts:
                        type_counts['URL'] = 0
                    type_counts['URL'] += 1
                else:
                    for t in column_types:
                        if t != 'URL':
                            if t not in type_counts:
                                type_counts[t] = 0
                            type_counts[t] += 1

            elif 'Text' in column_types:
                if ('False' == row or 'True' == row or 'false' == row or 'true' == row) and 'Boolean' in column_types:
                    if 'Boolean' not in type_counts:
                        type_counts['Boolean'] = 0
                    type_counts['Boolean'] += 1
                else:
                    if 'Text' not in type_counts:
                        type_counts['Text'] = 0
                    type_counts['Text'] += 1

            elif 'ItemList' in column_types:
                if 'ItemList' not in type_counts:
                        type_counts['ItemList'] = 0
                type_counts['ItemList'] += 1

            else:
                for column_type in column_types:
                    if column_type not in type_counts:
                        type_counts[column_type] = 0

                    type_counts[column_type] += 1
    
    return type_counts

In [ ]:
def get_type(file_name, column_name):
    
    if file_name in existing_english_tables:
        file = 'output-data/expanded-tables/' + file_name
    else:
        file = 'output-data/new-english-tables/' + file_name
        
    df = pd.read_json('data/stc-zip-files/' + file_name, compression='gzip', lines=True)
        
    if ':name' in column_name:
        column_name = column_name.split(':')[0]
    
    #Open table
    if column_name not in df.columns:
        #If new column check in new expanded tables for type
        df = pd.read_json(file, compression='gzip', lines=True)
        
    # Check the types of all rows if all column values not empty:
    if len(df[df[column_name].notna()][column_name].tolist()):
        column_types = column_cta[column_name].split(', ')
        # Count how many rows fall under a type
        type_counts = {}

        for row in df[df[column_name].notna()][column_name].tolist():

            #Check if row is a dictionary
            if isinstance(row, dict):
                type_counts = get_dict_type(row, column_types, column_name, type_counts)

            #If row is a list
            elif isinstance(row, list):
                type_counts = get_list_of_dict_type(row, column_types, column_name, type_counts)

            #If row is empty/null: add count to all types == Undefined type
            elif pd.isnull(row):
                for column_type in column_types:
                    if column_type not in type_counts:
                        type_counts[column_type] = 0
                    type_counts[column_type] += 1

            #Else if row is a string, number or datetime value
            else:
                type_counts = get_str_type(row, column_types, column_name, type_counts)

        #Choose the type where most of rows belong to:
        row_number = len(df[df[column_name].notna()][column_name].tolist()) #Total number of non null rows
        majority_rows = type_counts[max(type_counts, key=type_counts.get)] #How many rows with majority type?

        #Majority type
        if all(x==list(type_counts.values())[0] for x in list(type_counts.values())) and len(type_counts) > 1:
            majority_type = column_fallback[column_name]
        else:
            majority_type = max(type_counts, key=type_counts.get)

        return majority_type
        
    else:
        #Return None for empty columns
        return None

In [ ]:
def get_labels(table):
    
    try:
    
        class_name = table.split("_")[0] #Schema.org type
        columns = list(eval(tables[table]['column_name_and_density']).keys())
        columns_cpa_labels = {}
        columns_cta_labels = {}


        for column in columns:
            columns_cpa_labels[column] = column_cpa[column]
            
            column_before = column

            if column == 'name':
                column = class_name + 'name'

            #For CTA label of columns that can have multiple types: try to check from its original column in the tables what type it is
            if ', ' in column_cta[column]:     
                columns_cta_labels[column] = get_type(table, column)
            else:   
                columns_cta_labels[column_before] = column_cta[column]

        return [
            class_name, 
            table, 
            tables[table]['column_count'], 
            tables[table]['number_of_rows'], 
            tables[table]['overall_table_density'], 
            tables[table]['column_name_and_density'], 
            columns_cpa_labels, 
            columns_cta_labels
        ]
    
    except KeyError:
        print(table)


In [ ]:
table_names = list(tables.keys())
pool = multiprocessing.Pool(processes=30)
res = pool.map(get_labels, table_names)
pool.close()
pool.join()

In [ ]:
annotated_tables = pd.DataFrame(res, columns=['class', 'file_name', 'column_count', 'number_of_rows', 'overall_table_density', 'all_cols', 'rel_labels', 'type_labels'])
annotated_tables

In [ ]:
annotations_dict = annotated_tables.to_dict('records')

In [ ]:
rels = {}
types = {}
for row in annotations_dict:
    rel_labels = row['rel_labels']
    type_labels = row['type_labels']
    densities = eval(row['all_cols'])

    for col in densities:
        if 'name' in densities:
            if rel_labels[col] not in rels:
                rels[rel_labels[col]] = {}

            if row['file_name'] not in rels[rel_labels[col]]:
                rels[rel_labels[col]][row['file_name']] = {}

            rels[rel_labels[col]][row['file_name']][col] = densities[col]



        if type_labels[col] not in types:
            types[type_labels[col]] = {}


        if row['file_name'] not in types[type_labels[col]]:
            types[type_labels[col]][row['file_name']] = {}

        types[type_labels[col]][row['file_name']][col] = densities[col]
            

In [ ]:
#To calculate some statistics about each CTA/CPA label
rels_tab = []
types_tab = []

for rel in rels:
    c = 0
    for tab in rels[rel]:
        c += len(rels[rel][tab])
        
    rels_tab.append([rel, len(rels[rel]), c, rels[rel]])

for t in types:
    c = 0
    for tab in types[t]:
        c += len(types[t][tab])
        
    types_tab.append([t, len(types[t]), c, types[t]])

In [ ]:
cpa_statistics = pd.DataFrame(rels_tab, columns=['cpa_label', 'table_count', 'column_count', 'tables_and_densities'])
cta_statistics = pd.DataFrame(types_tab, columns=['cta_label', 'table_count', 'column_count', 'tables_and_densities'])

In [ ]:
#Manual corrections for CTA labels:
#Correct geo
list_tables_geo = ['LocalBusiness_village-hotels.co.uk_September2020.json.gz', 'Hotel_bikershotel.it_September2020.json.gz']

# Correct geo
for geo_tab in list_tables_geo:
    row = annotated_tables.loc[annotated_tables['file_name'] == geo_tab ]
    
    idx = row.index[0]
    
    annotated_tables.loc[annotated_tables['file_name'] == geo_tab ]['type_labels'][idx]['geo'] = 'geo'

# Correct author
row = annotated_tables.loc[annotated_tables['file_name'] == "Recipe_t2tea.com_September2020.json.gz" ]    
idx = row.index[0]
annotated_tables.loc[annotated_tables['file_name'] == 'Recipe_t2tea.com_September2020.json.gz' ]['type_labels'][idx]['author'] = 'Person'

# Fix recipeinstructions
row = annotated_tables.loc[annotated_tables['file_name'] == "Recipe_accessiblechef.com_September2020.json.gz" ]    
idx = row.index[0]
annotated_tables.loc[annotated_tables['file_name'] == 'Recipe_accessiblechef.com_September2020.json.gz' ]['type_labels'][idx]['recipeinstructions'] = 'ItemList'


In [ ]:
annotated_tables.to_csv('output-data/statistics/expanded_tables_annotations.csv', index=False)

In [ ]:
#Tables to choose from for CTA
annotated_tables.loc[annotated_tables['overall_table_density'] >= 70].to_csv('output-data/statistics/expanded_tables_annotations_cta.csv', index=False)

In [ ]:
#Tables to choose from for CPA (tables should have name column!)
file_names = []

for row in annotations_dict:    
    if 'name' in eval(row['all_cols']):
        file_names.append(row['file_name'])

In [ ]:
annotated_tables.loc[annotated_tables['file_name'].isin(file_names)].to_csv('output-data/statistics/expanded_tables_annotations_cpa.csv', index=False)

In [ ]:
cpa_statistics.to_csv('output-data/statistics/cpa_statistics.csv', index=False)
cta_statistics.to_csv('output-data/statistics/cta_statistics.csv', index=False)